# SPARK SQL


In [1]:
#Inicializacion de spark session y creacion de nuestra instancia
import findspark
findspark.init()
findspark.find()
import pyspark
from pyspark.sql import SparkSession

In [2]:
#para crear nuestro sesion de spark sql
spark = SparkSession.builder.appName('sesion3').getOrCreate()

In [3]:
spark #para ver informacion de nuestro sesion

In [4]:
#Para empezar a trabajar vamos a crear un json para importarlo y crear nuestra primera tabla
#De paso repasamos como crear un json a mano desde python..

import os
file = open("nombres.json",'w') #Le damos un nombre y w para que podamos escribir
file.write("{'Nombre':'Juan','Edad':34,'Ciudad':'Elche'}" + '\n')
file.write("{'Nombre':'Maria','Edad':23,'Ciudad':'Murcia'}"+'\n')
file.write("{'Nombre':'Laura','Edad':19,'Ciudad':'Alicante'}"+'\n')
file.write("{'Nombre':'Sandra','Edad':45,'Ciudad':'Alicante'}"+'\n')
file.write("{'Nombre':'Pedro','Edad':20,'Ciudad':'Elche'}"+'\n')

file.close()

#Si vamos a la carpeta veremos que se ha creado el fichero json


In [4]:
#Creamos el dataframe a partir del json que hemos creado:
mi_dataframe = spark.read.json('nombres.json')

In [5]:
mi_dataframe
#Vemos que se nos ha creado un dataframe con las columnas Ciudad, Edad, Nombre con los tipos de datos correspondientes,
#los tipos de dato los genera como spark cree, luego veremos como podemos cambiar esto si no nos cuadra el tipo de dato
#que se ha asignado automaticamente

DataFrame[Ciudad: string, Edad: bigint, Nombre: string]

In [6]:
#Podemos visualizar nuestro dataframe (en formato tabla) con show
mi_dataframe.show()
#Como vemos el formato es muy similar al de una tabla normal 

+--------+----+------+
|  Ciudad|Edad|Nombre|
+--------+----+------+
|   Elche|  34|  Juan|
|  Murcia|  23| Maria|
|Alicante|  19| Laura|
|Alicante|  45|Sandra|
|   Elche|null| Pedro|
+--------+----+------+



In [7]:
#podemos ver la escructura del dataframe:
mi_dataframe.printSchema()

root
 |-- Ciudad: string (nullable = true)
 |-- Edad: long (nullable = true)
 |-- Nombre: string (nullable = true)



In [8]:
#Podemos ver solo los nombres de las columnas:
mi_dataframe.columns
#Es un atributo, por lo que solo con columns ya nos muestra el valor

['Ciudad', 'Edad', 'Nombre']

In [9]:
#Podemos ver un resumen de estadisticas de la tabla con describe
mi_dataframe.describe()#... habria que usar show() como abajo

DataFrame[summary: string, Ciudad: string, Edad: string, Nombre: string]

In [31]:
#En este caso tenemos que usar show() para que nos muestre la informacion
mi_dataframe.describe().show()

+-------+--------+------------------+------+
|summary|  Ciudad|              Edad|Nombre|
+-------+--------+------------------+------+
|  count|       5|                 5|     5|
|   mean|    null|              28.2|  null|
| stddev|    null|11.122050170719426|  null|
|    min|Alicante|                19|  Juan|
|    max|  Murcia|                45|Sandra|
+-------+--------+------------------+------+



In [32]:
#Como vemos saca algunas estadisticas como el count de elementos, el maximo minimo, media...
#Vemos que nos saca el resultado por columna,el count es interesante ya que no tiene porqué un registro tener todos los datos
#por ejemplo si modificamos el json y le quitamos a Pedro la edad

In [33]:
#Para empezar a trabajar vamos a crear un json para importarlo y crear nuestra primera tabla
import os
file = open("nombres.json",'w')
file. write("{'Nombre':'Juan','Edad':34,'Ciudad':'Elche'}" + '\n')
file. write("{'Nombre':'Maria','Edad':23,'Ciudad':'Murcia'}"+'\n')
file. write("{'Nombre':'Laura','Edad':19,'Ciudad':'Alicante'}"+'\n')
file. write("{'Nombre':'Sandra','Edad':45,'Ciudad':'Alicante'}"+'\n')
file. write("{'Nombre':'Pedro','Ciudad':'Elche'}"+'\n')

file. close()

In [35]:
#Vemos que ahora en count de la edad nos muestra un 4
mi_dataframe.describe().show()

+-------+--------+------------------+------+
|summary|  Ciudad|              Edad|Nombre|
+-------+--------+------------------+------+
|  count|       5|                 4|     5|
|   mean|    null|             30.25|  null|
| stddev|    null|11.701139545645402|  null|
|    min|Alicante|                19|  Juan|
|    max|  Murcia|                45|Sandra|
+-------+--------+------------------+------+



# SCHEMA 
Modificación de los datos

In [42]:
#Como vemos los tipos de datos nos lo da spark por defecto, pero podemos modificarlos, para ello vamos a necesitar importar
#las siguientes clases las estructuras y los tipos de datos que necesitamos

from pyspark.sql.types import StructField,StructType,IntegerType,StringType


In [53]:
#vamos a crear la estructura que queremos para nuestro dataframe de antes
mi_estructura = [StructField('Nombre',StringType(),False),
              StructField('Edad',IntegerType(),True),
              StructField('Ciudad',StringType(),False)
             ]

In [54]:
#Como vemos hemos creado la escructura deseada, con los tipos de datos que queremos y los nulos que queremos, en nuestro caso, 
#por ejemplo nos interesaba que la edad pudiera ser nula, pero el nombre y ciudad no


In [55]:
#Ahora vamos a asignar nuestra estructura deseada al schema
mi_esquema = StructType(fields=mi_estructura)

In [56]:
#Creamos el dataframe igual que antes, pero ahora le pasaremos el esquema que hemos creado para que lo utilice
mi_dataframe = spark.read.json('nombres.json',schema=mi_esquema)

In [57]:
mi_dataframe.printSchema()

root
 |-- Nombre: string (nullable = true)
 |-- Edad: integer (nullable = true)
 |-- Ciudad: string (nullable = true)



In [59]:
#Destacar que cuando leemos de un fichero los datos siempre van con el nullable a true auqnue forcemos que sean False, spark
#no se fia en este sentido de que pueda ir un dato a nulo, entonces siempre permite nulos cuando leemos de fichero

In [10]:
mi_dataframe.show()

+--------+----+------+
|  Ciudad|Edad|Nombre|
+--------+----+------+
|   Elche|  34|  Juan|
|  Murcia|  23| Maria|
|Alicante|  19| Laura|
|Alicante|  45|Sandra|
|   Elche|null| Pedro|
+--------+----+------+

